<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px" />

# Excel Madness Lab!

_Author:_ Tim Book

## Our Mission
We work for a large supermarket chain, with stores in 10 major cities that happen to coincide with General Assembly campuses. However, this company's idea of a "database" is just a bunch of Excel spreadsheets! In order to analyze our data, we're going to need to process the existing data into a form we can use. **Our end goal is to have one csv per city.**

## Cleanup Duty!
It is a hard truth that data scientists spend a large majority of their time cleaning data. Data never arrives on our desks in exactly the format in which we want it, and it's up to us to transform it to a workable format.

Being good cleaning, moving, and reshaping data is in itself a valuable and employable job skill. If you follow these directions exactly, we will walk through constructing an automated process for processing data from this supermarket chain.

# Part I: Processing

### Step 1: Imports and the `os` library
We're going to import three libraries: numpy, pandas, and `os`.

In [101]:
# Import libraries here.
import numpy as np
import pandas as pd
import os.path

The `os` library is extremely useful for performing system commands from within Python. Let's get two pieces of overhead out of the way now:

1. Create an `output` folder using `os.mkdir()`
2. Create a variable called `files` that is the list of files in the `data` folder using `os.listdir()`

**WARNING:** The `os.mkdir()` function will give you an error if you try to make a folder that already exists!

In [102]:
# Create an output folder.
#os.mkdir('output')

# Create a files variable that contains all of our data files.


In [103]:
files = os.listdir('/Users/aryehgelfand/GADSI/labs/2.02-lab-excel-madness/data/')
files

['Jan 23.xlsx',
 'Jan 19.xlsx',
 'Jan 15.xlsx',
 'Jan 7.xlsx',
 'Jan 6.xlsx',
 'Jan 14.xlsx',
 'Jan 18.xlsx',
 'Jan 22.xlsx',
 'Jan 29.xlsx',
 'Jan 13.xlsx',
 'Jan 1.xlsx',
 'Jan 25.xlsx',
 'Jan 24.xlsx',
 'Jan 12.xlsx',
 'Jan 28.xlsx',
 'Jan 3.xlsx',
 'Jan 11.xlsx',
 'Jan 27.xlsx',
 'Jan 31.xlsx',
 'Jan 30.xlsx',
 'Jan 26.xlsx',
 'Jan 10.xlsx',
 'Jan 2.xlsx',
 'Jan 9.xlsx',
 'Jan 21.xlsx',
 'Jan 5.xlsx',
 'Jan 17.xlsx',
 'Jan 16.xlsx',
 'Jan 4.xlsx',
 'Jan 20.xlsx',
 'Jan 8.xlsx']

### Step 2: Process one data frame
It looks like we have data for the month of January. 31 files of 10 sheets each! Luckily they are all in the same format. So let's read just one in and process that. It might be helpful to open one up in your spreadsheet viewer of choice first (Excel, Numbers, Sheets, etc.)

In [104]:
import xlrd

In [105]:
# Read in data from your city from January 1st.
jan_1 = pd.read_excel('/Users/aryehgelfand/GADSI/labs/2.02-lab-excel-madness/data/Jan 1.xlsx')

In [106]:
jan_1

,prodcode,price_eu,weight_kg,quantity
0,4159,2.048141,8.812961,138
1,4062,2.728485,9.331372,184
2,4404,2.182498,8.937798,491
3,4412,2.342694,7.067682,218
4,4131,1.802839,9.551503,391
...,...,...,...,...
124,4796,1.827676,7.922745,444
125,4225,2.463229,5.839095,258
126,4134,0.920152,9.225959,395
127,3302,1.510998,2.667668,168


### Step 2a: Convert to 'Merican columns
For whatever reason, our data are stored in euros and kilograms. Create `price_usd` and `weight_lb` columns. There are 2.2 pounds per kilogram, and 1.1 dollars per euro.

In [107]:
jan_1['price_usd'] = jan_1['price_eu']*1.1 # convert to usd

In [108]:
jan_1['weight_lb'] = jan_1['weight_kg']*2.2 # convert to lbs

In [109]:
jan_1

,prodcode,price_eu,weight_kg,quantity,price_usd,weight_lb
0,4159,2.048141,8.812961,138,2.252955,19.388515
1,4062,2.728485,9.331372,184,3.001334,20.529018
2,4404,2.182498,8.937798,491,2.400748,19.663156
3,4412,2.342694,7.067682,218,2.576964,15.548899
4,4131,1.802839,9.551503,391,1.983123,21.013306
...,...,...,...,...,...,...
124,4796,1.827676,7.922745,444,2.010444,17.430039
125,4225,2.463229,5.839095,258,2.709552,12.846008
126,4134,0.920152,9.225959,395,1.012167,20.297110
127,3302,1.510998,2.667668,168,1.662098,5.868870


### Step 2b: Merge in product names
You'll notice we also have a `plu-codes.csv` file containing actual product names matched up against their price lookup (PLU) codes. Let's merge these product names onto our Jan 1 data.
* _Hint 1:_ What kind of merge is this? Right, left, inner, outer, etc.?
* _Hint 2:_ Pay special attention to column names!

In [110]:
plu = pd.read_csv("plu-codes.csv")

In [111]:
plu.rename(columns={'plu_code':'prodcode'},inplace=True)


In [112]:
jan_1.dtypes

prodcode       int64
price_eu     float64
weight_kg    float64
quantity       int64
price_usd    float64
weight_lb    float64
dtype: object

In [113]:
data = pd.merge(jan_1, plu, on= 'prodcode', how='left')

In [114]:
data.head()

,prodcode,price_eu,weight_kg,quantity,price_usd,weight_lb,product
0,4159,2.048141,8.812961,138,2.252955,19.388515,Onion
1,4062,2.728485,9.331372,184,3.001334,20.529018,Cucumber
2,4404,2.182498,8.937798,491,2.400748,19.663156,Peach
3,4412,2.342694,7.067682,218,2.576964,15.548899,Pear
4,4131,1.802839,9.551503,391,1.983123,21.013306,Apple (Fuji)


### Step 2c: Drop unnecessary columns
We've created some extraneous columns. Drop the old price and weight columns, as well as any redundant columns.

In [115]:
data.drop('weight_kg', axis=1, inplace=True)# drop eu weight
data.drop('price_eu', axis=1, inplace=True)# drop eu pricing

### Step 2d: Add the date
Simply create a new `date` column that is the date this data was collected. For example, if this is from `Jan 1.xlsx`, this column should be full of `Jan 1`.

In [116]:
data['date'] = 'Jan 1'
data.head()

,prodcode,quantity,price_usd,weight_lb,product,date
0,4159,138,2.252955,19.388515,Onion,Jan 1
1,4062,184,3.001334,20.529018,Cucumber,Jan 1
2,4404,491,2.400748,19.663156,Peach,Jan 1
3,4412,218,2.576964,15.548899,Pear,Jan 1
4,4131,391,1.983123,21.013306,Apple (Fuji),Jan 1


### Step 3: Write a function that conducts all of Step 2
This function should import a **filename and a city name** and return a fully processed DataFrame. That is, the function should:
1. Read in the data from the given file and city.
1. Create USD and pound columns.
1. Merge in product names.
1. Drop unnecessary columns.
1. Add a date column

In [117]:
def process_data(file, city):
    fn = file
    xl = pd.ExcelFile(fn)
    xl.sheet_names
    for sh in xl.sheet_names:
         df = xl.parse(sh)# copied this section of the code from https://stackoverflow.com/questions/42887663/reading-multiple-tabs-from-excel-in-different-dataframes
    df['price_usd'] = df['price_eu']*1.1
    df['weight_lb'] = df['weight_kg']*2.2
    plu = pd.read_csv("plu-codes.csv")
    plu.rename(columns={'plu_code':'prodcode'},inplace=True)
    df = pd.merge(df, plu, on= 'prodcode', how='left')
    df.drop('weight_kg', axis=1, inplace=True)
    df.drop('price_eu', axis=1, inplace=True)
    s1 = str(file).strip('/Users/aryehgelfand/GADSI/labs/2.02-lab-excel-madness/data/') # starts with file name, strips '.xlsx' 
    date = s1.replace(" ","") # removes space for better naming
    #This section was adpated from Jordan Weiss slacked shared code
    df['date'] = date
    return df


Test your function out on a new file and city!

In [118]:
bos = process_data('/Users/aryehgelfand/GADSI/labs/2.02-lab-excel-madness/data/Jan 1.xlsx', 'Boston')

### Step 4: Process all of January's data
For each spreadsheet, process the data and store the resulting DataFrame in one big list. **You only need to do this for your city!**

* _Hint 1:_ A listcomp would make this whole step one line of code!
* _Hint 2:_ You've already made that `files` variable to help you here.

### Step 4: Process all of January's data
For each spreadsheet, process the data and store the resulting DataFrame in one big list. **You only need to do this for your city!**

* _Hint 1:_ A listcomp would make this whole step one line of code!
* _Hint 2:_ You've already made that `files` variable to help you here.

In [119]:
ny_df = [process_data(('data/{}'.format(file)), "New York") for file in files]

In [120]:
pd_ny = pd.concat(ny_df)# Step # 5

### Step 6: Do this for all cities, write data
Here's the big one. For each city, process and the data as in steps 3-5, and then write the data to our `output` folder. Below is a dictionary of city name to desired output file name.

Before writing your DataFrame, do the following:
* Add a `city` column
* Reorder the columns into the following order:


| city | date | product | prodcode | quantity | weight_lb | price_usd |
|---|---|---|---|---|---|---|

* _Hint:_ You can reorder DataFrame columns simply by writing over your DataFrame with itself, but specifying the specific column order with `.loc`. For example:

`print(df)`

| b | c | a |
|---|---|---|
| 1 | 2 | 3 |

`df = df.loc[:, ["a", "b", "c"]]`

`print(df)`

| a | b | c |
|---|---|---|
| 3 | 1 | 2 |


In [121]:
city_dict = {
    "Atlanta": "atl.csv",
    "Austin": "atx.csv",
    "Boston": "bos.csv",
    
    "Chicago": "chi.csv",
    "Denver": "den.csv",
    "Los Angeles": "lax.csv",
    "New York": "nyc.csv",
    "San Francisco": "sf.csv",
    "Seattle": "sea.csv",
    "Washington, DC": "dc.csv"
}



In [122]:
# Loop through city_dict and carry out Step 6 here.


for city,out in city_dict.items():
    full_list = [process_data(('data/{}'.format(file)), city) for file in files]
    city_df = pd.concat(full_list)
    city_df["city"] = city
    city_df = city_df.loc[:, ["city", "date", "product", "prodcode", "quantity", "weight_lb", "price_usd"]]    
    city_df.to_csv("/Users/aryehgelfand/GADSI/labs/2.02-lab-excel-madness/output/"+out)
    
  #The keys of city_dict can serve as the sheet name.
#values of city_dict can serve as filename in output
    

# If done correctly, this cell could take almost a minute to run!

In [123]:


#converting sheet names to city name

# Part II: Checking our answers 
In steps very similar to the ones conducted above...
1. Loop through the files we just wrote to `output`, and read them in, collecting them all in one list
1. Combine all of those DataFrames into one large DataFrame
1. For each city, find the mean `quantity`, `weight_lb`, and `price_usd`.

If you've done everything correct, your answer should look exactly like this:

![](imgs/correct-output.png)

In [233]:
path = ("/Users/aryehgelfand/GADSI/labs/2.02-lab-excel-madness/output/")

In [234]:
folder_name = path
file_type = 'csv'
seperator =','
data = pd.concat([pd.read_csv(f, sep=seperator) for f in glob.glob(folder_name + "/*."+file_type)],ignore_index=True)

In [245]:
data.head()



,city,date,product,prodcode,quantity,weight_lb,price_usd
0,Los Angeles,Jan23,Lemon,4958,197,11.363450,1.946762
1,Los Angeles,Jan23,Cucumber,4062,313,17.817644,1.910404
2,Los Angeles,Jan23,Eggplant,4081,243,16.143858,3.101128
3,Los Angeles,Jan23,Grapes,4272,136,5.372800,2.295056
4,Los Angeles,Jan23,Apple (Gala),4134,179,14.775389,1.789376


In [246]:
data.drop('Unnamed: 0', axis=1, inplace=True)

KeyError: "['Unnamed: 0'] not found in axis"

In [242]:
def means(data):    
    for city in city_dict.items():
        return pd == (np.mean(data[data['city'] == city]['quantity'])),
        (np.mean(data[data['city'] == city]['weight_lb'])),
        (np.mean(data[data['city'] == city]['price_usd']))


# Part III (BONUS): Get this process production-ready!
_This part of the lab is optional, but very highly recommended, as the skills developed in this part are extremely common in industry._

For this step, we're going to take this whole process and put it into a production-ready Python script. **ABSOLUTELY NONE OF THIS STEP SHOULD TAKE PLACE IN A JUPYTER NOTEBOOK! PRODUCTIONALIZED ETL (_"Extract, Transform, Load"_) CODE DOES NOT TAKE PLACE IN A JUPYTER NOTEBOOK!**

The instructions are simple: As conducted in this lab, read, transform, and export the data in our Excel files into .csv files. The code should be in a `.py` file and executable from the command line. Here are some hints and tips to guide you:

### Hints, tips, and tricks:
* A good place to start is with the code you've already written. In this notebook, you can click `File > Download as > Python (.py)` to export as a `.py` file. Most of this exercise then comes down to you cleaning this file. (There will be a lot to clean).
* Remember `os.mkdir()` will throw an error if the folder you're trying to make already exists. Maybe you should check to see if it already exists. If it already exists, what should you do? (Remember that `.csv` can be overwritten with no problem.) The functions that can help you with this are all in the `os` library.
* Remember to follow all of the Python best practices we've already learned:
    - All import statements should go at the top of your script.
    - Comment your code. Comments shouldn't explain _what_ code does, but _why_ the code does this.
    - Keep your code DRY (don't repeat yourself) as opposed to WET (write everything twice). All constants should be variables that only need to be changed once. All code should be bottled into functions so you only need to fix it once.
* Make sure not to hardcode "Jan" anywhere. The point is that this code can be run throughout the lifetime of this supermarket's project, which is likely months or years. Keep your code so that if you get February data, you only need to change one tiny piece of the script (probably a file path).